In [12]:
import os

import pandas as pd
import numpy as np
from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")

In [20]:
base_dir = "/Users/szhan/Projects/tsimpute/viz/"
prefix = "sisu42_into_sisu3.imputation.results"

in_sts_csv_file = base_dir + "sts" + "." + prefix + "." + "pass_2" + "." + "csv"
out_sts_png_file = base_dir + "sts" + "." + prefix + "." + "pass_2" + "." + "png"

in_beagle_csv_file = base_dir + "beagle" + "." + prefix + "." + "csv"
out_beagle_png_file = base_dir + "beagle" + "." + prefix + "." + "png"


In [21]:
df_sts = pd.read_csv(in_sts_csv_file, comment="#")
df_beagle = pd.read_csv(in_beagle_csv_file, comment="#")

In [15]:
shared_site_pos = np.sort(list(set(df_sts["position"].to_numpy()) & set(df_beagle["position"].to_numpy())))

print(f"{df_sts.shape[0]}")
print(f"{df_beagle.shape[0]}")
print(f"{len(shared_site_pos)}")

97114
96075
95024


In [16]:
df_sts = df_sts[np.isin(df_sts["position"], shared_site_pos)]
df_beagle = df_beagle[np.isin(df_beagle["position"], shared_site_pos)]

df_sts = df_sts[["ref_minor_allele_freq", "iqs"]].dropna(axis=0)
df_beagle = df_beagle[["ref_minor_allele_freq", "iqs"]].dropna(axis=0)

In [ ]:
def plot_results(df, method, out_png_file, max_maf=0.50, dpi=100):
    values = np.vstack([df["ref_minor_allele_freq"], df["iqs"]])
    kernel = stats.gaussian_kde(values)
    x = kernel(np.vstack([df["ref_minor_allele_freq"], df["iqs"]]))

    fig, ax = plt.subplots(figsize=(10, 10))

    ax.set_title(
        f"{method}: from SiSu v4.2 into SiSu v3.0",
        size=45
    )
    ax.set_xlim([0, max_maf])
    ax.set_ylabel("IQS", size=40)
    ax.set_xlabel("MAF", size=40)
    ax.tick_params(axis='both', which='major', labelsize=35)

    g = sns.scatterplot(
        y="iqs",
        x="ref_minor_allele_freq",
        data=df,
        c=x,
        cmap="viridis",
        #x_jitter=True,
        ax=ax
    );

    g.get_figure().savefig(out_png_file, dpi=dpi)

In [ ]:
plot_results(df_sts, "Tree sequence", out_sts_png_file, dpi=1_000)

In [ ]:
plot_results(df_beagle, "BEAGLE", out_beagle_png_file, dpi=1_000)

In [27]:
# Rare
df_sts_rare = df_sts[df_sts["ref_minor_allele_freq"] <= 0.01]
df_beagle_rare = df_beagle[df_beagle["ref_minor_allele_freq"] <= 0.01]

# Uncommon
df_sts_uncommon = df_sts[df_sts["ref_minor_allele_freq"] > 0.01]
df_sts_uncommon = df_sts_uncommon[df_sts_uncommon["ref_minor_allele_freq"] <= 0.05]

df_beagle_uncommon = df_beagle[df_beagle["ref_minor_allele_freq"] > 0.01]
df_beagle_uncommon = df_beagle_uncommon[df_beagle_uncommon["ref_minor_allele_freq"] <= 0.05]

# Common
df_sts_common = df_sts[df_sts["ref_minor_allele_freq"] > 0.05]
df_beagle_common = df_beagle[df_beagle["ref_minor_allele_freq"] > 0.05]

In [ ]:
min_iqs = 0.90

print(f"Percent sites with IQS >= {min_iqs}")
print("ALL")
score_sts_all = float(df_sts[df_sts["iqs"] >= min_iqs].shape[0]) / float(df_sts.shape[0])
score_beagle_all = float(
    df_beagle[df_beagle["iqs"] >= min_iqs].shape[0]) / float(df_beagle.shape[0])
print(f"STS: {round(score_sts_all, 4)}")
print(f"BGL: {round(score_beagle_all, 4)}")
print(f"DIF: {round(score_beagle_all - score_sts_all, 4)}")

print("RARE")
score_sts_rare = df_sts_rare[df_sts_rare["iqs"] >= min_iqs].shape[0] / float(df_sts_rare.shape[0])
score_beagle_rare = df_beagle_rare[df_beagle_rare["iqs"] >= min_iqs].shape[0] / float(df_beagle_rare.shape[0])
print(f"STS: {round(score_sts_rare, 4)}")
print(f"BGL: {round(score_beagle_rare, 4)}")
print(f"DIF: {round(score_beagle_rare - score_sts_rare, 4)}")

print("UNCOMMON")
score_sts_uncommon = df_sts_uncommon[df_sts_uncommon["iqs"] >= min_iqs].shape[0] / float(df_sts_uncommon.shape[0])
score_beagle_uncommon = df_beagle_uncommon[df_beagle_uncommon["iqs"] >= min_iqs].shape[0] / float(df_beagle_uncommon.shape[0])
print(f"STS: {round(score_sts_uncommon, 4)}")
print(f"BGL: {round(score_beagle_uncommon, 4)}")
print(f"DIF: {round(score_beagle_uncommon - score_sts_uncommon, 4)}")

print("COMMON")
score_sts_common = df_sts_common[df_sts_common["iqs"] >= min_iqs].shape[0] / float(df_sts_common.shape[0])
score_beagle_common = df_beagle_common[df_beagle_common["iqs"] >= min_iqs].shape[0] / float(df_beagle_common.shape[0])
print(f"STS: {round(score_sts_common, 4)}")
print(f"BGL: {round(score_beagle_common, 4)}")
print(f"DIF: {round(score_beagle_common - score_sts_common, 4)}")